In [1]:
import pandas as pd
import minsearch as ms
from openai import OpenAI

In [2]:
# Cargamos los datos
df = pd.read_csv('../DATASETS/faq_sacmex.csv')

In [3]:
df

,id,pregunta,respuesta,document
0,d7c4ce5eda85cd602edc71a3f29193e0,¿Dónde puedo reportar una fuga de agua?,Debes reportarla al organismo operador de agua...,faq_sacmex
1,c6b1388a0227cae6d6045a1dc7dd82c5,¿Qué debo hacer si veo una fuga de agua en la ...,"En la Ciudad de México, debes reportarla al 55...",faq_sacmex
2,b9e493259e0b9fb9f94e4f4d66ab8ded,¿Qué debo hacer si veo una fuga de agua en mi ...,Lo primero que debes hacer es cortar el sumini...,faq_sacmex
3,5229ec6562607f2de830a7826a099964,¿Qué debo hacer si mi consumo de agua parece h...,"Si no tienes fuga de agua en tu casa, puedes t...",faq_sacmex
4,9055d52c36234595f93430ca60610b82,¿Cuánto tiempo tarda en repararse una fuga de ...,"Después de supervisar los materiales, maquinar...",faq_sacmex
5,34f49a09c09f373422893abead2bad4e,¿Dónde puedo reportar una fuga de agua?,Para reportar una fuga de agua en la Ciudad de...,faq_sacmex
6,221e2b757cee67a9babd822666eae6b1,¿Cómo reportar fugas de agua en CDMX?,Para realizar cualquier reporte por fuga en la...,faq_sacmex
7,deb8c6e4cf607cabc9fd72178b45daae,¿Qué otro de reportes puedo realizar?,Tambien puedes reportar:\n- Brotes de aguas ne...,faq_sacmex
8,977ff53e8650679587f99ee537c8b095,¿Qué debo tener en cuenta para levantar mi rep...,Para levantar un reporte debes tener en cuenta...,faq_sacmex
9,e118293c4450a72349aa684baa54f5eb,¿Puedo reportar fugas en unidades habitacionales?,"No, las fugas en unidades habitacionales son r...",faq_sacmex


In [ ]:
index = ms.Index(
    text_fields=['pregunta', 'respuesta'],
    keyword_fields=['document']
)

In [4]:
documents = df.to_dict(orient='records')
documents

[{'id': 'd7c4ce5eda85cd602edc71a3f29193e0',
  'pregunta': '¿Dónde puedo reportar una fuga de agua?',
  'respuesta': 'Debes reportarla al organismo operador de agua potable y alcantarillado de tu localidad.',
  'document': 'faq_sacmex'},
 {'id': 'c6b1388a0227cae6d6045a1dc7dd82c5',
  'pregunta': '¿Qué debo hacer si veo una fuga de agua en la calle?',
  'respuesta': 'En la Ciudad de México, debes reportarla al 55 5654 3210 para que una brigada del Sistema de Aguas de la Ciudad de México acuda a repararla.',
  'document': 'faq_sacmex'},
 {'id': 'b9e493259e0b9fb9f94e4f4d66ab8ded',
  'pregunta': '¿Qué debo hacer si veo una fuga de agua en mi casa?',
  'respuesta': 'Lo primero que debes hacer es cortar el suministro de agua, ya sea con el grifo de cierre o la válvula de cierre, que generalmente se encuentran en el baño o la cocina.\xa0',
  'document': 'faq_sacmex'},
 {'id': '5229ec6562607f2de830a7826a099964',
  'pregunta': '¿Qué debo hacer si mi consumo de agua parece haberse elevado?',
  're

In [ ]:
for doc in documents:
    for key in doc:
        if isinstance(doc[key], float):
            doc[key] = str(doc[key])

index.fit(documents)

In [ ]:
def search(query):
    boost = {'pregunta': 3.0, 'respuesta': 0.5}
    # boost = {}
    number_of_results = 5

    results = index.search(
        query,
        filter_dict={},
        boost_dict=boost,
        num_results=number_of_results
    )
    return results

In [5]:
documents[0]

{'id': 'd7c4ce5eda85cd602edc71a3f29193e0',
 'pregunta': '¿Dónde puedo reportar una fuga de agua?',
 'respuesta': 'Debes reportarla al organismo operador de agua potable y alcantarillado de tu localidad.',
 'document': 'faq_sacmex'}

In [6]:
entry_template = """
pregunta: {pregunta}
respuesta: {respuesta}
""".strip()

prompt_template = """
Eres un agente chatbot que ayuda a levantar reportes de fugas de agua en la SACMEX. Responde a la PREGUNTA de abajo basado en el CONTEXTO. Usa el CONTEXTO para dar una repuesta precisa.

PREGUNTA: {pregunta}

CONTEXTO: {respuesta}
"""


def build_prompt(query, results):
    context = ""
    for doc in results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(pregunta=query, respuesta=context).strip()
    return prompt

In [7]:
query = documents[0]['pregunta']

In [ ]:
search_results = search(query)
prompt = build_prompt(query, search_results)
print(prompt)

In [8]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [ ]:
def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                'role': 'system',
                'content': 'Eres un agente útil.'
            },
            {
                'role': 'user',
                'content': prompt
            }
        ]
    )
    return response.choices[0].message.content

In [ ]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt, 'llama3.2:3b')
    return answer

In [ ]:
query = '¿Dónde puedo reportar una fuga de agua?'
answer = rag(query)
print(answer)

## Indentificando el tipo de Solicitud

In [11]:
query = documents[0]['pregunta']
query

'¿Dónde puedo reportar una fuga de agua?'

In [99]:
entry_template = """
- {tipo}: {solicitud}\n
""".strip()

prompt_template = """
Eres un agente de SACMEX.

INSTRUCCIONES ESTRICTAS:
- Analiza cuidadosamente la SOLICITUD
- Genera ÚNICAMENTE un JSON con dos campos: "tipo_solicitud" y "solicitud"
- NO incluyas información adicional ni estructuras anidadas
- El "tipo_solicitud" debe ser UNO de estos: "pregunta", "nuevo_reporte", "status_reporte"
- La "solicitud" es el texto original del usuario
- Sin texto adicional, solo el JSON en una línea exacta

FORMATO REQUERIDO:
{{"tipo_solicitud": "categoria", "solicitud": "texto_completo"}}

EJEMPLOS:
{contexto}

REGLA PRINCIPAL:
- Elige ÚNICAMENTE UN tipo de solicitud que coincida EXACTAMENTE
- Si la solicitud es sobre CÓMO, DÓNDE o INFORMACIÓN para realizar un reporte o cuales son los telefonos o redes sociales, usa "pregunta"
- Si la solicitud contiene palabras clave como "levantar", "reportar", "fuga", usa "nuevo_reporte"
- Si la solicitud menciona "estatus" o "reporte" específico, usa "status_reporte"
- Si no hay coincidencia exacta, usa "otro" por defecto

SOLICITUD: {solicitud}

JSON DEFINITIVO:"""


def build_prompt(query, results):
    context = ""
    for doc in results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(solicitud=query, contexto=context).strip()
    return prompt

In [104]:
tipos = [{
        "solicitud": "¿Como realizo un reporte?",
        "tipo": "pregunta"
    },{
        "solicitud": "¿Dónde puedo levantar un reporte?",
        "tipo": "pregunta"
    },{
        "solicitud": "¿Donde puedo levantar un reporte?",
        "tipo": "pregunta"
    },
    {
        "solicitud": "Sabes cuales son los telefonos de la sacmex",
        "tipo": "pregunta"
    },
    {
        "solicitud": "Hola, me gustaria levantar reporte de fuga de agua en la calle orquidea entre calle cda 5 de mayo y vicente guerrero, en san pedro martir, aca en Tlalpan",
        "tipo": "nuevo_reporte"
    },
    {
        "solicitud": "quisiera reportar una fuga de fuga de agua",
        "tipo": "nuevo_reporte"
    },
    {
        "solicitud": "Hola, me gustaria conocer el estatus de mi reporte",
        "tipo": "status_reporte"
    },
    {
        "solicitud": "cual es el estatus del reporte XXX-YYY-UUU",
        "tipo": "status_reporte"
    },
]

In [100]:
def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                'role': 'assistant',
                'content': 'Eres un agente útil.'
            },
            {
                'role': 'user',
                'content': prompt
            }
        ],
        temperature=0.7
    )
    return response.choices[0].message.content

In [105]:
prompt = build_prompt("Me gustaria reportar una fuga en la calle bugambilias, en la colonia condechi, en la gustavo a madero", tipos)
print(prompt)

Eres un agente de SACMEX.

INSTRUCCIONES ESTRICTAS:
- Analiza cuidadosamente la SOLICITUD
- Genera ÚNICAMENTE un JSON con dos campos: "tipo_solicitud" y "solicitud"
- NO incluyas información adicional ni estructuras anidadas
- El "tipo_solicitud" debe ser UNO de estos: "pregunta", "nuevo_reporte", "status_reporte"
- La "solicitud" es el texto original del usuario
- Sin texto adicional, solo el JSON en una línea exacta

FORMATO REQUERIDO:
{"tipo_solicitud": "categoria", "solicitud": "texto_completo"}

EJEMPLOS:
- pregunta: ¿Como realizo un reporte?

- pregunta: ¿Dónde puedo levantar un reporte?

- pregunta: ¿Donde puedo levantar un reporte?

- pregunta: Sabes cuales son los telefonos de la sacmex

- nuevo_reporte: Hola, me gustaria levantar reporte de fuga de agua en la calle orquidea entre calle cda 5 de mayo y vicente guerrero, en san pedro martir, aca en Tlalpan

- nuevo_reporte: quisiera reportar una fuga de fuga de agua

- status_reporte: Hola, me gustaria conocer el estatus de mi 

In [96]:
answer = llm(prompt, 'llama3.2:3b')
answer

'{"tipo_solicitud": "nuevo_reporte", "solicitud": "Me gustaria reportar una fuga en la calle bugambilias, en la colonia condechi, en la gustavo a madero"}'

In [106]:
prompt = build_prompt(query, tipos)
print(prompt)

Eres un agente de SACMEX.

INSTRUCCIONES ESTRICTAS:
- Analiza cuidadosamente la SOLICITUD
- Genera ÚNICAMENTE un JSON con dos campos: "tipo_solicitud" y "solicitud"
- NO incluyas información adicional ni estructuras anidadas
- El "tipo_solicitud" debe ser UNO de estos: "pregunta", "nuevo_reporte", "status_reporte"
- La "solicitud" es el texto original del usuario
- Sin texto adicional, solo el JSON en una línea exacta

FORMATO REQUERIDO:
{"tipo_solicitud": "categoria", "solicitud": "texto_completo"}

EJEMPLOS:
- pregunta: ¿Como realizo un reporte?

- pregunta: ¿Dónde puedo levantar un reporte?

- pregunta: ¿Donde puedo levantar un reporte?

- pregunta: Sabes cuales son los telefonos de la sacmex

- nuevo_reporte: Hola, me gustaria levantar reporte de fuga de agua en la calle orquidea entre calle cda 5 de mayo y vicente guerrero, en san pedro martir, aca en Tlalpan

- nuevo_reporte: quisiera reportar una fuga de fuga de agua

- status_reporte: Hola, me gustaria conocer el estatus de mi 

In [103]:
query

'¿Dónde puedo reportar una fuga de agua?'

In [107]:
answer = llm(prompt, 'llama3.2:3b')
answer

'{"tipo_solicitud": "pregunta", "solicitud": "D\\u00f1nde puedo reportar una fugua de agua?"}'

```json
{"tipo_solicitud": "pregunta", "solicitud": "D\\u00f1nde puedo reportar una fugua de agua?"}
```

In [109]:
json.loads(answer)

{'tipo_solicitud': 'pregunta',
 'solicitud': 'Dñnde puedo reportar una fugua de agua?'}

In [66]:
import json

if not answer.strip().endswith('}'):
    answer += '}'

In [110]:
prompt = build_prompt("Me gustaria saber el estatus del reporte FFF-1112-4423", tipos)
print(prompt)

Eres un agente de SACMEX.

INSTRUCCIONES ESTRICTAS:
- Analiza cuidadosamente la SOLICITUD
- Genera ÚNICAMENTE un JSON con dos campos: "tipo_solicitud" y "solicitud"
- NO incluyas información adicional ni estructuras anidadas
- El "tipo_solicitud" debe ser UNO de estos: "pregunta", "nuevo_reporte", "status_reporte"
- La "solicitud" es el texto original del usuario
- Sin texto adicional, solo el JSON en una línea exacta

FORMATO REQUERIDO:
{"tipo_solicitud": "categoria", "solicitud": "texto_completo"}

EJEMPLOS:
- pregunta: ¿Como realizo un reporte?

- pregunta: ¿Dónde puedo levantar un reporte?

- pregunta: ¿Donde puedo levantar un reporte?

- pregunta: Sabes cuales son los telefonos de la sacmex

- nuevo_reporte: Hola, me gustaria levantar reporte de fuga de agua en la calle orquidea entre calle cda 5 de mayo y vicente guerrero, en san pedro martir, aca en Tlalpan

- nuevo_reporte: quisiera reportar una fuga de fuga de agua

- status_reporte: Hola, me gustaria conocer el estatus de mi 

In [111]:
answer2 = llm(prompt, 'llama3.2:3b')
answer2

'{"tipo_solicitud": "status_reporte", "solicitud": "Me gustaria saber el estatus del reporte FFF-1112-4423"}'

## Extracción de datos

In [112]:
tipos = [
    {
        "solicitud": "Hola, me gustaria levantar reporte de fuga de agua en la calle orquidea entre calle cda 5 de mayo y vicente guerrero, en san pedro martir, aca en Tlalpan",
        "tipo": "nuevo_reporte"
    },
    {
        "solicitud": "quisiera reportar una fuga de fuga de agua en la calle orquidea casi esquina con cda 5 de mayo, en san pedro martir, en tlalpan",
        "tipo": "nuevo_reporte"
    },
    {
        "solicitud": "Hola, hay un problema de aguas negras en la cda 5 de mayo casi llegando al panteon, colonia san pedro martir, en la delegación tlalpan",
        "tipo": "nuevo_reporte"
    }
]

In [119]:
dataentry_template = """
- {solicitud}\n
""".strip()

prompt_template = """
Eres un agente de SACMEX especializado en extracción de información de direcciones de nuevos reportes sobre el agua.

INSTRUCCIONES PRECISAS:
- Extrae ÚNICAMENTE la información de ubicación de la fuga
- Genera un JSON con los siguientes campos:
    * "calle_principal": nombre de la calle principal
    * "entre_calles": calles cercanas (si se mencionan)
    * "colonia": nombre de la colonia
    * "delegacion": nombre de la delegación o alcaldía
    * "referencias_adicionales": cualquier detalle extra de ubicación
- Sin texto adicional, solo el JSON en una línea exacta.

FORMATO REQUERIDO:
{{
    "calle_principal": "Nombre de la calle",
    "entre_calles": ["Calle 1", "Calle 2"],
    "colonia": "Nombre de la colonia",
    "delegacion": "Nombre de la delegación",
    "referencias_adicionales": "Detalles extra"
}}

EJEMPLOS:
{contexto}

SOLICITUD: {solicitud}

JSON DEFINITIVO:"""



def build_prompt(query, results):
    context = ""
    for doc in results:
        context = context + dataentry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(solicitud=query, contexto=context).strip()
    return prompt

In [120]:
prompt = build_prompt("Hola, me gustaria levantar reporte de fuga de agua en la calle orquidea entre calle cda 5 de mayo y vicente guerrero, en san pedro martir, aca en Tlalpan", tipos)
print(prompt)

Eres un agente de SACMEX especializado en extracción de información de direcciones de nuevos reportes sobre el agua.

INSTRUCCIONES PRECISAS:
- Extrae ÚNICAMENTE la información de ubicación de la fuga
- Genera un JSON con los siguientes campos:
    * "calle_principal": nombre de la calle principal
    * "entre_calles": calles cercanas (si se mencionan)
    * "colonia": nombre de la colonia
    * "delegacion": nombre de la delegación o alcaldía
    * "referencias_adicionales": cualquier detalle extra de ubicación
- Sin texto adicional, solo el JSON en una línea exacta.

FORMATO REQUERIDO:
{
    "calle_principal": "Nombre de la calle",
    "entre_calles": ["Calle 1", "Calle 2"],
    "colonia": "Nombre de la colonia",
    "delegacion": "Nombre de la delegación",
    "referencias_adicionales": "Detalles extra"
}

EJEMPLOS:
- Hola, me gustaria levantar reporte de fuga de agua en la calle orquidea entre calle cda 5 de mayo y vicente guerrero, en san pedro martir, aca en Tlalpan

- quisiera r

In [121]:
answer_1 = llm(prompt, 'llama3.2:3b')
print(answer_1)

{
    "calle_principal": "Orquídea",
    "entre_calles": ["CDA 5 DE MAYO", "Vicente Guerrero"],
    "colonia": "San Pedro Martir",
    "delegacion": "Tlalpan",
    "referencias_adicionales": "Aca en Tlalpan"


In [124]:
if not answer_1.strip().endswith('}'):
    answer_1 += '}'

print(json.loads(answer_1))

{'calle_principal': 'Orquídea', 'entre_calles': ['CDA 5 DE MAYO', 'Vicente Guerrero'], 'colonia': 'San Pedro Martir', 'delegacion': 'Tlalpan', 'referencias_adicionales': 'Aca en Tlalpan'}
